Notebook to construct videos that visualize salmon ID annotation

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import json
import random
random.seed(327364)

import os
import sys
sys.path.append(os.path.split(os.getcwd())[0] + '\\helpers')

import keybox_utils as ku
from draw_utils import draw_text

In [2]:
## Specify paths
labelme_path =  'C:\\Users\\espen\\Documents\\work\\PhD\\papers\\salmon_tracking\\data\\datasets\\with_global_salmon_IDs\\CS_val\\'
out_video_name = 'ID_assignment_video.avi'

In [3]:
def create_ID_annotation_video(labelme_path, out_video_name, end_frame, IW = 4242, IH = 4242):
    ''' 
    Function to create a video of the salmon ID annotation. All frames in the raw video will be included in the generated video. 
    The annotated frames will be drawn with bounding boxes around the fish, their ID written at the corner of the box, and a dotted line showing past fish localizations for the given ID.
    Args:
        labelme_path: Path to the labelme annotations.
        out_video_name: Path to the output video.
        end_frame: Last frame in the raw video to include in the generated video.
    File operations:
        Saving a video at the location specified by the out_video_name.
    
    '''
    colors = [(random.randint(0,255), random.randint(0,255), random.randint(0,255)) for i in range(300)]
    label_names = os.listdir(labelme_path + 'labels')
    img_names = os.listdir(labelme_path + 'images')
    writer = cv2.VideoWriter(out_video_name,  
                        cv2.VideoWriter_fourcc(*'MJPG'), 
                        5, (IW,IH)) 
    frame_num = 0
    location_history = {k:[] for k in range(300)}
    for img_name in img_names:
        if frame_num > end_frame:
            break
        img = cv2.imread(labelme_path + '\\images\\' + img_name)
        if img_name.split('.')[0] + '.json' in label_names:
            label_name = img_name.split('.')[0] + '.json'
            with open(labelme_path + '\\labels\\' + label_name, 'r') as f:
                label = json.load(f)
                for item in label['shapes']:
                    if item['label'] == 'salmon':
                        x, y, w, h = ku.xyxy2xywh(np.array(item['points']).flatten())
                        cv2.rectangle(img, (int(item['points'][0][0]), int(item['points'][0][1])), (int(item['points'][1][0]), int(item['points'][1][1])), (0, 0, 255) , 2)
                        draw_text(img, str(item['group_id']), pos = (min(int(item['points'][0][0]), int(item['points'][1][0])), min(int(item['points'][0][1]), int(item['points'][1][1]))))
                        location_history[int(item['group_id'])].append((int(x), int(y)))
                        lp = None
                        for center in location_history[int(item['group_id'])]:
                            cv2.circle(img, center, 10, colors[int(item['group_id'])], -1)
                            if lp != None:
                                cv2.line(img, lp, center, colors[int(item['group_id'])], 3)
                            lp = center
        writer.write(img)
        frame_num = frame_num + 1
create_ID_annotation_video(labelme_path, out_video_name, 50)